In [1]:
import spacy
import re
from random import shuffle
from spacy.util import minibatch, compounding
from tqdm import tqdm

# optimizer = nlp.begin_training(device=0) # Para usar a GPU device=0

# https://github.com/explosion/thinc#quickstart  # instalar think para rodar na gpu

# https://spacy.io/api/cli

In [2]:
class Trainer:
    
    def __init__(self):
        
        self.TD = [
        
            ("O dr. Everton Tomalok da JustiÇa foi convidado a vir assinar a papelada.", {
                'entities': [(25, 34, 'ORG'), (6, 21, 'PER')]
            }),

            ("O dr. Arnaldo da Silva da JustiÇa, é um bom advogado", {
                'entities': [(26, 35, 'ORG')]
            }),

            ("É um lugar?", {
                'entities': []
            }),

            ("JustiÇa é uma empresa.", {
                'entities': [(0, 9, 'ORG')]
            }),

            ("JustiÇa é legal.", {
                'entities': [(0, 9, 'ORG')]
            }),

            ('O dr. Júlio da Silva da JustiÇa Ltda. é legal.', {
                'entities': [(24, 37, 'ORG'), (6, 20, 'PER')]
            }),

            ('O dr. Lucas Bulhões Santos trabalha na OAB', {
                'entities': [(2, 15, 'PER'), (28, 34, 'ORG')]
            }),

            ('O dr. Lucas da Empreiteira.', {
                'entities': [(6, 11, 'PER'), (15, 26, 'ORG')]
            }),

            ('O dr. Fabiano do Federzoni.', {
                'entities': [(6, 11, 'PER'), (15, 24, 'ORG')]
            })
            
        ]
        
        self.nlp = spacy.load('pt_core_news_sm')
        self.ner = self.nlp.get_pipe('ner')


    def add_label(self, new_label):
                
        self.ner.add_label(new_label)
        
        
    def tag_prhase(self, string, parser):
        
        finder = re.finditer(parser, string)

        try:
            
            prox = next(finder)
            del finder
            
            init_char = prox.start()
            end_char = prox.end()
            
            return init_char, end_char
            
        except StopIteration:
            
            return None, None
        
    
    def update_train_data(self, tuple_to_add):
        
        assert isinstance(tuple_to_add, tuple), """
        Formato ACEITO: 
            ('Eu irei dia 10/12/2018.', {
            'entities': [(12, 22, 'DATE')]
            })
        """
        
        assert isinstance(tuple_to_add[0], str), """
        Formato ACEITO: 
            ('Eu irei dia 10/12/2018.', {
            'entities': [(12, 22, 'DATE')]
            })
        """
        
        assert isinstance(tuple_to_add[1], dict), """
        Formato ACEITO: 
            ('Eu irei dia 10/12/2018.', {
            'entities': [(12, 22, 'DATE')]
            })
        """
        
        assert 'entities' in tuple_to_add[1], """
        Formato ACEITO: 
            ('Eu irei dia 10/12/2018.', {
            'entities': [(12, 22, 'DATE')]
            })
        """
        
        self.TD.append(tuple_to_add)
        
    
    def train(self, epochs=100):
        
        other_pipes = [pipe for pipe in self.nlp.pipe_names if pipe != 'ner']
        
        optimizer = self.nlp.entity.create_optimizer() 

        with tqdm(initial=0, total=epochs, 
                  bar_format='{percentage:3.0f}%|{bar}|{n_fmt}/{total_fmt} > Time: {elapsed} <> {desc}') as pbar:

            with self.nlp.disable_pipes(*other_pipes):  # only train NER
                
                for _ in range(epochs):

                    shuffle(self.TD)
                    losses = {}

                    batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
                    for batch in batches:
                        texts, annotations = zip(*batch)
                        self.nlp.update(texts, annotations, sgd=optimizer, drop=0.5, losses=losses)

                    pbar.set_description(f'Loss: {losses["ner"]}')
                    pbar.update(1)
                    

trainer = Trainer()
TRAIN_DATA = trainer.TD


In [3]:
trainer.tag_prhase('Eu nasci dia 15/08/2018.','15/08/2018')

(13, 23)

In [4]:
trainer.tag_prhase('O JusBrasil não vai abrir hoje.', 'JusBrasil')

(2, 11)

In [5]:
label = 'DATA'

data_to_add = [
                ('O Saito também é uma empresa.', 
                 {'entities': [(2, 7, 'ORG')]}
                ),
                ('O Espetão é uma empresa.', 
                 {'entities': [(2, 9, 'ORG')]}
                ),
                ('Eu nasci dia 15/08/2018.', {
                    'entities': [(13, 23, label)]
                }),
                ('Eu irei dia 10/12/2018.', {
                    'entities': [(12, 22, label)]
                }),
                ('Ontem foi 10/02/2017.', {
                    'entities': [(10, 20, label)]
                }),
                ('O Espetão é uma churrascaria.', {
                    'entities': [(2, 9, 'ORG')]
                }),
               ]

trainer.add_label(label)

for data in data_to_add:
    trainer.update_train_data(data)

In [6]:
trainer.TD

[('O dr. Everton Tomalok da JustiÇa foi convidado a vir assinar a papelada.',
  {'entities': [(25, 34, 'ORG'), (6, 21, 'PER')]}),
 ('O dr. Arnaldo da Silva da JustiÇa, é um bom advogado',
  {'entities': [(26, 35, 'ORG')]}),
 ('É um lugar?', {'entities': []}),
 ('JustiÇa é uma empresa.', {'entities': [(0, 9, 'ORG')]}),
 ('JustiÇa é legal.', {'entities': [(0, 9, 'ORG')]}),
 ('O dr. Júlio da Silva da JustiÇa Ltda. é legal.',
  {'entities': [(24, 37, 'ORG'), (6, 20, 'PER')]}),
 ('O dr. Lucas Bulhões Santos trabalha na OAB',
  {'entities': [(2, 15, 'PER'), (28, 34, 'ORG')]}),
 ('O dr. Lucas da Empreiteira.',
  {'entities': [(6, 11, 'PER'), (15, 26, 'ORG')]}),
 ('O dr. Fabiano do Federzoni.',
  {'entities': [(6, 11, 'PER'), (15, 24, 'ORG')]}),
 ('O Saito também é uma empresa.', {'entities': [(2, 7, 'ORG')]}),
 ('O Espetão é uma empresa.', {'entities': [(2, 9, 'ORG')]}),
 ('Eu nasci dia 15/08/2018.', {'entities': [(13, 23, 'DATA')]}),
 ('Eu irei dia 10/12/2018.', {'entities': [(12, 22, 'DATA'

In [7]:
trainer.train(100)

100%|██████████|100/100 > Time: 00:38 <> Loss: 1.8116777353286015: 


In [8]:
def print_ents(doc):
    for ent in doc.ents:
        print('ENT: ', ent.text, ' - LABEL: ', ent.label_)
        
        if ent.label_ != 'DATA':
            for d in ent:
                if d.pos_ == 'PROPN' or d.pos_ == 'NOUN':
                    print('         - ', d.text,'  | POS: ', d.pos_)

            print('==========\n')
        else:
            print()

In [18]:
test_text = 'O dr. Bruno Silva da JusTiça é legal.'
doc = trainer.nlp(test_text)

print_ents(doc)

ENT:  Bruno Silva  - LABEL:  PER
         -  Bruno   | POS:  PROPN
         -  Silva   | POS:  PROPN

ENT:  JusTiça é  - LABEL:  ORG
         -  JusTiça   | POS:  PROPN



In [20]:
test_text = 'O dr. Bruno que trabalha na Peijim Ltda. esta aqui.'
doc = trainer.nlp(test_text)

print_ents(doc)

ENT:  Bruno que  - LABEL:  PER
         -  Bruno   | POS:  PROPN

ENT:  Peijim Ltda.  - LABEL:  ORG
         -  Peijim   | POS:  PROPN
         -  Ltda   | POS:  PROPN



In [11]:
test_text = 'O dr. Gustavo Gusmão da Silva da Jussara, irá falar.'
doc = trainer.nlp(test_text)

print_ents(doc)

ENT:  Gustavo Gusmão da Silva  - LABEL:  PER
         -  Gustavo   | POS:  PROPN
         -  Gusmão   | POS:  PROPN
         -  Silva   | POS:  PROPN

ENT:  Jussara  - LABEL:  ORG
         -  Jussara   | POS:  PROPN



In [12]:
test_text = 'O advogado Augusto Carriere que é o advogado do Federzoni, irá falar no dia 10/12/2018.'
doc = trainer.nlp(test_text)

print_ents(doc)

ENT:  Augusto Carriere  - LABEL:  PER
         -  Augusto   | POS:  PROPN
         -  Carriere   | POS:  PROPN

ENT:  Federzoni  - LABEL:  PER
         -  Federzoni   | POS:  PROPN

ENT:  10/12/2018  - LABEL:  DATA



In [13]:
test_text = 'O representante dr. Augusto, advogado do Bradesco, me disse isso hoje de manhã.'
doc = trainer.nlp(test_text)

print_ents(doc)

ENT:  Augusto,  - LABEL:  PER
         -  Augusto   | POS:  PROPN

ENT:  Bradesco  - LABEL:  ORG
         -  Bradesco   | POS:  PROPN



In [14]:
test_text = 'Irei ao Espetão no dia 13/10/2030.'
doc = trainer.nlp(test_text)

print_ents(doc)

ENT:  Espetão  - LABEL:  ORG
         -  Espetão   | POS:  PROPN

ENT:  13/10/2030  - LABEL:  DATA



In [15]:
test_text = 'O JusBrasil vai lançar uma nova ferramenta.'

doc = trainer.nlp(test_text)

print_ents(doc)

ENT:  JusBrasil  - LABEL:  ORG
         -  JusBrasil   | POS:  PROPN



In [16]:
test_text = 'O tribunal não terá audiências durante a manhã.'

doc = trainer.nlp(test_text)

print_ents(doc)